
# Information about the system & environment

In [18]:
import os
import platform
import sys

import psutil

Note that `psutil` is not in Python's standard library.

## Operating system

Using the `platform` module, it is easy to obtain information on the platform the Python interpreter is running on.

Information about the machine name:

In [9]:
platform.node()

'ubuntu'

The architecture and hardware:

In [16]:
platform.processor()

'x86_64'

In [13]:
sys.byteorder

'little'

In [19]:
os.cpu_count()

1

In [20]:
os.sched_getaffinity(0)

{0}

The operating system:

In [10]:
platform.system()

'Linux'

In [14]:
platform.release()

'4.15.0-38-generic'

In [15]:
platform.version()

'#41-Ubuntu SMP Wed Oct 10 10:59:38 UTC 2018'

In [17]:
platform.linux_distribution()

('debian', 'buster/sid', '')

In [6]:
platform.platform()

'Linux-4.15.0-38-generic-x86_64-with-debian-buster-sid'

## Numerics

The properties of floating point numbers can be obtained easily from the `sys.floatinfo` object.

The largest floating point value that can be represented, the smallest positive non-zero value:

In [4]:
print(sys.float_info.max, sys.float_info.min)

1.7976931348623157e+308 2.2250738585072014e-308


The number of significant digits of a floating point value:

In [2]:
sys.float_info.dig

15

## Processes

Detailed information is available on the processes running on the system.

In [26]:
for process in psutil.process_iter():
    if 'gjb' in process.username():
        cpu_times = process.cpu_times()
        thread_str = f'threads: {process.num_threads()}'
        cpu_str = f'user: {cpu_times.user}, sys: {cpu_times.system}'
        print(f'{process.pid}: {process.name()} ({thread_str}, {cpu_str})')

2089: systemd (threads: 1, user: 0.16, sys: 0.13)
2090: (sd-pam) (threads: 1, user: 0.0, sys: 0.0)
2103: gnome-keyring-daemon (threads: 4, user: 0.39, sys: 0.11)
2107: gdm-x-session (threads: 3, user: 0.0, sys: 0.01)
2109: Xorg (threads: 2, user: 60.06, sys: 84.17)
2116: dbus-daemon (threads: 1, user: 0.54, sys: 0.3)
2120: gnome-session-binary (threads: 4, user: 0.12, sys: 0.25)
2227: ssh-agent (threads: 1, user: 0.05, sys: 0.15)
2229: at-spi-bus-launcher (threads: 4, user: 0.0, sys: 0.02)
2234: dbus-daemon (threads: 1, user: 0.12, sys: 0.05)
2237: at-spi2-registryd (threads: 3, user: 0.59, sys: 0.52)
2250: gvfsd (threads: 3, user: 0.04, sys: 0.01)
2255: gvfsd-fuse (threads: 6, user: 0.0, sys: 0.0)
2272: gnome-shell (threads: 10, user: 1727.11, sys: 268.63)
2281: pulseaudio (threads: 3, user: 0.49, sys: 7.16)
2292: ibus-daemon (threads: 3, user: 21.9, sys: 6.38)
2296: ibus-dconf (threads: 4, user: 0.0, sys: 0.01)
2298: ibus-x11 (threads: 3, user: 0.19, sys: 0.09)
2300: ibus-portal (thr

CPU times are cumulative over the process' life time.

In [28]:
for process in psutil.process_iter():
    if process.cpu_times().user > 2.5:
        print(f'{process.name()}: {process.cpu_times().user}')

vmtoolsd: 18.13
snapd: 4.69
dbus-daemon: 3.19
dockerd: 45.21
docker-containerd: 26.17
gnome-shell: 30.3
Xwayland: 20.83
packagekitd: 23.36
gsd-color: 6.5
Xorg: 60.43
gnome-shell: 1731.97
ibus-daemon: 22.12
gsd-color: 6.18
vmtoolsd: 43.48
nautilus-desktop: 5.89
ibus-engine-simple: 7.62
gnome-software: 9.81
gnome-terminal-server: 7.03
ZMQbg/1: 10.69
chrome: 138.51
chrome: 44.74
chrome: 532.83
chrome: 3.39
chrome: 26.69
zeal: 5.41
chrome: 13.82


It is easy to kill processes, so you might want to be careful.

In [31]:
name = 'zeal'
killed_pids = []
for process in psutil.process_iter():
    if name == process.name():
        print(f'killing {name}...')
        killed_pids.append(process.pid)
        process.kill()
print('killed: ', ', '.join(map(str, killed_pids)))

killing zeal...
killed:  18898


## Performance

The `psutil` module makes quite some interesting statistics related to system performance available. This can be useful when writing monitoring tools.

The cumulative times for user, nice, system and so on are readily available.

In [32]:
psutil.cpu_times()

scputimes(user=2992.29, nice=67.76, system=919.06, idle=58311.66, iowait=76.19, irq=0.0, softirq=12.07, steal=0.0, guest=0.0, guest_nice=0.0)

Memory usage can be queried.

In [33]:
psutil.virtual_memory()

svmem(total=4112015360, available=1358479360, percent=67.0, used=2459013120, free=474349568, active=1601912832, inactive=935960576, buffers=344289280, cached=834363392, shared=19922944, slab=262770688)

Disk I/O measures such as the total IOP and read/write sizes in byte are easy to obain.

In [35]:
psutil.disk_io_counters()

sdiskio(read_count=462764, write_count=114006, read_bytes=5862454272, write_bytes=3719344128, read_time=328092, write_time=831264, read_merged_count=72280, write_merged_count=176461, busy_time=213560)

Network I/O can similarly be monitored.

In [49]:
psutil.net_io_counters()

snetio(bytes_sent=21116518, bytes_recv=86326034, packets_sent=61886, packets_recv=102453, errin=0, errout=0, dropin=0, dropout=0)

Disk usage for all partitions can be queried.

In [48]:
for partition in psutil.disk_partitions():
    mountpoint = partition.mountpoint
    if 'snap' not in mountpoint:
        print(f'{mountpoint}: {psutil.disk_usage(mountpoint).percent:.1f}')

/: 77.5
/var/lib/docker/aufs: 77.5
